In [2]:
import numpy as np
import snorkel
from snorkel.labeling import labeling_function
from snorkel.labeling import LabelingFunction
from snorkel.labeling import PandasLFApplier
from snorkel.labeling import LFAnalysis
import re
from snorkel.labeling.model import MajorityLabelVoter
import json
import pandas as pd
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [3]:
%load_ext autoreload
%autoreload 2
import label_improve as li

In [4]:
# Loading the data 
dataset_name = "claude9"
idx_to_label = json.load(open(f"../weak_datasets/{dataset_name}/label.json"))
label_to_idx = {l:i for i,l in idx_to_label.items()}
valid_df = li.data_to_df(json.load(open(f"../weak_datasets/{dataset_name}/valid.json", "r")))
train_df = li.data_to_df(json.load(open(f"../weak_datasets/{dataset_name}/train.json", "r")))
test_df = li.data_to_df(json.load(open(f"../weak_datasets/{dataset_name}/test.json", "r")))

# Sample a dev set to help seed ideas for LFs
dev_df = train_df.sample(250, random_state=123)

In [8]:
test_df = test_df.sample(250, random_state=205).drop(columns=[ 'label', 'weak_labels'])
 

In [11]:
test_df.to_csv('./llm/banking_eval.csv')

In [9]:
# Get rid of the "weak_labels" column
dev_df = dev_df.drop(columns=["weak_labels"])
dev_df.head()

,text,label
5376,you must use this address to opt-out : \n,8
4283,"content includes , without limitation , all us...",8
4124,spotify may terminate the agreements or suspen...,1
2579,reporting copyright and other ip violations \n,8
2929,"( a ) loss of profit , \n",8


In [8]:
# Save the dataset to csv
dev_df.to_csv(f"./llm/claude9.csv", index=False)

In [9]:
li.analysis_LFs_with_weak_labels(test_df,9)

Test Coverage: 1.0
acuracy for the not abstains
0.9133398247322297
acuracy for all
0.9120077783179388


In [12]:
keywords = {
    "0": ["liability", "damages", "loss", "indemnification", "compensation"],
    "1": ["terminate", "termination", "cancel", "end", "suspend"],
    "2": ["change", "modify", "alter", "amend", "update"],
    "3": ["remove", "delete", "take down", "withdraw", "retract"],
    "4": ["contract", "agreement", "terms", "conditions", "binding"],
    "5": ["law", "legal", "governing law", "jurisdiction", "regulation"],
    "6": ["jurisdiction", "court", "venue", "forum", "authority"],
    "7": ["arbitration", "dispute", "resolution", "mediator", "settlement"],
    "8": ["reporting", "copyright", "violation", "address", "opt-out"]
}

# replace the key 0-8 with the actual labels
{
    "0": "Limitation of liability",
    "1": "Unilateral termination",
    "2": "Unilateral change",
    "3": "Content removal",
    "4": "Contract by using",
    "5": "Choice of law",
    "6": "Jurisdiction",
    "7": "Arbitration",
    "8": "Other"
}

keywords = {
    "Limitation of liability": ["liability", "damages", "loss", "indemnification", "compensation"],
    "Unilateral termination": ["terminate", "termination", "cancel", "end", "suspend"],
    "Unilateral change": ["change", "modify", "alter", "amend", "update"],
    "Content removal": ["remove", "delete", "take down", "withdraw", "retract"],
    "Contract by using": ["contract", "agreement", "terms", "conditions", "binding"],
    "Choice of law": ["law", "legal", "governing law", "jurisdiction", "regulation"],
    "Jurisdiction": ["jurisdiction", "court", "venue", "forum", "authority"],
    "Arbitration": ["arbitration", "dispute", "resolution", "mediator", "settlement"],
    "Other": ["reporting", "copyright", "violation", "address", "opt-out"]
}

 
    

In [13]:
keywords_by_idx = {int(label_to_idx[k]):v for k,v in keywords.items()}
lfs = li.keywords_to_LFs(keywords_by_idx)
li.analysis_LFs(lfs, test_df, 77)

  0%|          | 0/2057 [00:00<?, ?it/s]

100%|██████████| 2057/2057 [00:00<00:00, 5450.49it/s]


Test Coverage: 0.6003889158969373
acuracy for the not abstains
0.146524064171123
acuracy for all
0.06660184735051045


,j,Polarity,Coverage,Overlaps,Conflicts,Correct,Incorrect,Emp. Acc.,Conflict Ratio
lf_liability,0,[0],0.034030,0.024793,0.021877,23,47,0.328571,0.642857
lf_damages,1,[0],0.019446,0.016043,0.013126,12,28,0.300000,0.675000
lf_loss,2,[0],0.020418,0.014584,0.011667,17,25,0.404762,0.571429
lf_indemnification,3,[0],0.002431,0.001458,0.000486,1,4,0.200000,0.200000
lf_compensation,4,[0],0.005348,0.001945,0.001945,5,6,0.454545,0.363636
lf_terminate,5,[1],0.029169,0.025280,0.021390,28,32,0.466667,0.733333
lf_termination,6,[1],0.022363,0.015557,0.014584,2,44,0.043478,0.652174
lf_cancel,7,[1],0.030141,0.017015,0.012640,0,62,0.000000,0.419355
lf_end,8,[1],0.127370,0.071463,0.062713,14,248,0.053435,0.492366
lf_suspend,9,[1],0.008264,0.008264,0.004375,10,7,0.588235,0.529412
